### Load packages and settings

In [1]:
# packages
import time
import pandas as pd
from datetime import date
import numpy as np

# Decision Tree, Random Forest & XGBoost packages
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# TensorFlow packages
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
#from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
#from sklearn.pipeline import Pipeline

# settings
start_time=time.time()
yrs_list = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']
first_year = '2005'


### Load master dataframes from the individual years of College Football Stats

In [2]:
def concat_mult_ref_tables(filename, yrs):
    """Return a dataframe that concatenates all 
    files across a list of years, with the year set as a key
    """
    # create a list to store the dfs
    df_list = []
    
    for yr in yrs: 
        temp_df = None   #clear out the df
        temp_df = pd.read_csv('./input/' + str(yr) + '/' + filename + '.csv')  #read in the file
        df_list.append(temp_df)
        
    final_df = pd.concat(df_list, keys=yrs)
    #final_df.rename_axis(['Season', 'Ix'])
    
    return final_df

# Create master dataframes with all the available years
conference_mstr = concat_mult_ref_tables('conference', yrs_list)
drive_mstr = concat_mult_ref_tables('drive', yrs_list)
game_statistics_mstr = concat_mult_ref_tables('game-statistics', yrs_list)
game_mstr = concat_mult_ref_tables('game', yrs_list)
kickoff_return_mstr = concat_mult_ref_tables('kickoff-return', yrs_list)
kickoff_mstr = concat_mult_ref_tables('kickoff', yrs_list)
pass_mstr = concat_mult_ref_tables('pass', yrs_list)
play_mstr = concat_mult_ref_tables('play', yrs_list)
player_game_statistics_mstr = concat_mult_ref_tables('player-game-statistics', yrs_list)
player_mstr = concat_mult_ref_tables('player', yrs_list)
punt_return_mstr = concat_mult_ref_tables('punt-return', yrs_list)
punt_mstr = concat_mult_ref_tables('punt', yrs_list)
reception_mstr = concat_mult_ref_tables('reception', yrs_list)
rush_mstr = concat_mult_ref_tables('rush', yrs_list)
stadium_mstr = concat_mult_ref_tables('stadium', yrs_list)
team_game_statistics_mstr = concat_mult_ref_tables('team-game-statistics', yrs_list)
team_mstr = concat_mult_ref_tables('team', yrs_list)



### Produce a dataset that simply tells us who won each game

In [3]:
# create some new useful datasets
tmp_col_list = ['Team Code', 'Game Code', 'Points']
team_points = team_game_statistics_mstr[tmp_col_list]

# produce a new dataset called game_results_mstr that tells us who won each game
game_results_mstr = game_mstr
game_results_mstr['Visit Points'] = None
game_results_mstr['Home Points'] = None
game_results_mstr['Home Team Winner'] = None
game_results_mstr['Final Spread'] = None
game_results_mstr['Total Points'] = None
for index, row in game_results_mstr.iterrows():
    # find the visiting team's points
    visit_pts_ix = team_points.index[(team_points['Team Code']==row['Visit Team Code']) 
                    & (team_points['Game Code']==row['Game Code'])].tolist()
    visit_pts = team_points.loc[visit_pts_ix[0]].at['Points']
    # find the home team's points
    home_pts_ix = team_points.index[(team_points['Team Code']==row['Home Team Code']) 
                    & (team_points['Game Code']==row['Game Code'])].tolist()
    home_pts = team_points.loc[home_pts_ix[0]].at['Points']
    # calculate the spread and total points
    final_spread = abs(home_pts - visit_pts)
    total_pts = home_pts + visit_pts
    
    # save all values to the game_results dataset
    game_results_mstr.loc[index, 'Visit Points'] = visit_pts
    game_results_mstr.loc[index, 'Home Points'] = home_pts
    game_results_mstr.loc[index, 'Final Spread'] = final_spread
    game_results_mstr.loc[index, 'Total Points'] = total_pts
    if home_pts > visit_pts:
        game_results_mstr.loc[index, 'Home Team Winner'] = 1
    else:
        game_results_mstr.loc[index, 'Home Team Winner'] = 0

In [4]:
game_results_mstr.head()

Game Code        Date  Visit Team Code  Home Team Code  \
2005 0   86016420050901  09/01/2005               86             164   
     1  128064820050901  09/01/2005              128             648   
     2  204014020050901  09/01/2005              204             140   
     3  295075420050901  09/01/2005              295             754   
     4  428071920050901  09/01/2005              428             719   

        Stadium Code  Site  Visit Points  Home Points  Home Team Winner  \
2005 0            25  TEAM             0           38                 1   
     1           113  TEAM            15           24                 1   
     2            97  TEAM            26           28                 1   
     3           138  TEAM            26           38                 1   
     4           105  TEAM            41           10                 0   

        Final Spread  Total Points  
2005 0            38            38  
     1             9            39  
     2             2            54  
     3            12            64  
     4            31            51

### Create the master dataset for use in training and validating our game winner model

In [5]:
# Create a dataset that shows each game, by team, and the result 
# essentially double the size of the game_results set by breaking out winners and losers
home_cols = ['Game Code', 'Date', 'Home Team Code', 'Visit Team Code', 'Home Team Winner', 'Site']
away_cols = ['Game Code', 'Date', 'Visit Team Code', 'Home Team Code', 'Home Team Winner', 'Site']
new_cols = ['Game Code', 'Date', 'Team Code', 'Opp Code', 'Won', 'Game_Home']

def flip_winner(i):
    return int(not i)

def site_code(text):
    if text == 'TEAM':
        return 1
    else:
        return 0

# Create two dfs full of team specific results
home_df = game_results_mstr[home_cols]
home_df.columns = new_cols
#home_df['Home'] = 1
home_df['Game_Home'] = home_df['Game_Home'].apply(site_code)

away_df = game_results_mstr[away_cols]
away_df['Home Team Winner'] = away_df['Home Team Winner'].apply(flip_winner)
away_df.columns = new_cols
#away_df['Home'] = 0
away_df['Game_Home'] = 0

# Concatenate the dfs into one final df
gametime_master = pd.concat([home_df, away_df])

# Convert the Date column to Datetime dtype
gametime_master['Date'] = pd.to_datetime(gametime_master['Date'], infer_datetime_format=True)

gametime_master.head()

/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

Game Code       Date  Team Code  Opp Code  Won  Game_Home
2005 0   86016420050901 2005-09-01        164        86    1          1
     1  128064820050901 2005-09-01        648       128    1          1
     2  204014020050901 2005-09-01        140       204    1          1
     3  295075420050901 2005-09-01        754       295    1          1
     4  428071920050901 2005-09-01        719       428    0          1

In [6]:
gametime_master.tail()

Game Code       Date  Team Code  Opp Code  Won  Game_Home
2013 843  147051820140103 2014-01-03        147       518    1          0
     844  521043420140103 2014-01-03        521       434    0          0
     845  736028820140104 2014-01-04        736       288    1          0
     846   30004720140105 2014-01-05         30        47    1          0
     847   37023420140106 2014-01-06         37       234    0          0

### Write some functions to extract pre-game knowledge and add them to our dataset
#### Team Knowledge (Record, Conf Record, Total Pts, Total Passing Yds, Total Rush Yds, etc)
#### Opponent Knowledge (Same)
#### Concentric Knowledge (previous overlapping opponents)

In [ ]:
first_year = '2005'

def get_season_str_yr(gamedate):
    """ Takes the date of a game and 
    returns the season year as a string"""
    if gamedate.month == 1:  # if this is a bowl game
        str_year = str(gamedate.year - 1)
    else:
        str_year = str(gamedate.year)
    return str_year

def season_wins_to_date(team_code, date):
    """Given a team and date, this function returns the number season wins
    up to, but not including, that date.  If this is the first game of the season
    it returns the number of wins from last season"""
    # account for bowl games that occur in next calendar year
    str_year = get_season_str_yr(date)
    # locate the full season for this team
    team_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
    games_to_date = team_season[team_season['Date'] < date]
    games = games_to_date.shape[0]
    # account for first game of the season - use last year unless we don't have it
    if ((games == 0) & (str_year != first_year)):
        str_year = str(date.year - 1)
        
        # Handle errors when there is no last season
        try:
            last_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
            wins = last_season['Won'].sum()
        except KeyError:
            wins = 0
        
    else:
        wins = games_to_date['Won'].sum()
    return wins
    
def season_losses_to_date(team_code, date):
    """Given a team and date, this function returns the number season losses
    up to, but not including, that date.  If this is the first game of the season
    it returns the number of losses from last season"""
    # account for bowl games that occur in next calendar year
    str_year = get_season_str_yr(date)
    # locate the full season for this team and calculate losses
    team_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
    games_to_date = team_season[team_season['Date'] < date]
    games = games_to_date.shape[0]
    # account for first game of the season - use last year unless we don't have it
    if ((games == 0) & (str_year != first_year)):
        str_year = str(date.year - 1)
        # Handle errors when there is no last season
        try:
            last_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
            games = last_season.shape[0]
            wins = last_season['Won'].sum()
            losses = games - wins
        except KeyError:
            losses = 0
    else:        
        wins = games_to_date['Won'].sum()
        losses = games - wins
    return losses
    
def season_record_to_date(team_code, date):
    """Given a team and date, this function returns the season win percentage as a float
    up to, but not including, that date.  If this is the first game of the season
    it returns the percentage from last season"""
    # account for bowl games that occur in next calendar year
    str_year = get_season_str_yr(date)
    # locate the full season for this team and calculate wins
    team_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
    games_to_date = team_season[team_season['Date'] < date]
    games = games_to_date.shape[0]
    # account for first game of the season - use last year unless we don't have it
    if ((games == 0) & (str_year != first_year)):
        str_year = str(date.year - 1)
        # Handle errors when there is no last season
        try:
            last_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]
            games = last_season.shape[0]
            wins = last_season['Won'].sum()
            # don't allow to divide by zero
            if games > 0:
                win_perc = wins / games
            else:
                win_perc = 0
        except KeyError:
            win_perc = 0
    elif ((games == 0) & (str_year == first_year)):
        return 0
    else:
        wins = games_to_date['Won'].sum()
        win_perc = wins / games
    
    return round(win_perc, 3)

def conf_record_to_date(team_code, date):
    """Given a team and date, this function returns the season win percentage against 
    conference teams as a float up to, but not including, that date.  If this is the first 
    conference game of the season it returns the win percentage from last season"""
    # account for bowl games that occur in next calendar year
    str_year = get_season_str_yr(date)
    # locate the conference games for this team and calculate wins
    team_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year][gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]['Game_Conf'] == 1]
    games_to_date = team_season[team_season['Date'] < date]
    games = games_to_date.shape[0]
    # account for first game of the season - use last year unless we don't have it
    if ((games == 0) & (str_year != first_year)):
        str_year = str(date.year - 1)
        # Handle errors when there is no last season
        try:
            last_season = gametime_master[gametime_master['Team Code'] == team_code].loc[str_year][gametime_master[gametime_master['Team Code'] == team_code].loc[str_year]['Game_Conf'] == 1]
            games = last_season.shape[0]
            wins = last_season['Won'].sum()
            # don't allow to divide by zero
            if games > 0:
                win_perc = wins / games
            else:
                win_perc = 0
        except KeyError:
            win_perc = 0
    elif ((games == 0) & (str_year == first_year)):
        return 0
    else:
        wins = games_to_date['Won'].sum()
        win_perc = wins / games
    
    return round(win_perc, 3)



def in_conf_game(team_code1, team_code2, yr):
    conf1 = team_mstr[team_mstr['Team Code'] == team_code1].loc[yr].iloc[0]['Conference Code']
    conf2 = team_mstr[team_mstr['Team Code'] == team_code2].loc[yr].iloc[0]['Conference Code']
    if conf1 == conf2:
        return 1
    else:
        return 0
    
def get_norm_stadium_capacity(gamecode):
    """ Given a stadium code and gamedate
    return the normalized value of the stadium capacity"""
    date = pd.Timestamp(game_mstr[game_mstr['Game Code'] == gamecode]['Date'].values[0])
    str_year = get_season_str_yr(date)
    stadium_code = game_mstr[game_mstr['Game Code'] == gamecode]['Stadium Code'].values[0]
    max_stadium = max(stadium_mstr['Capacity'].tolist())
    capacity = stadium_mstr[stadium_mstr['Stadium Code'] == stadium_code].loc[str_year].Capacity.tolist()[0]
    norm_val = int(capacity) / int(max_stadium)
    return norm_val
    

### Add game specific stats to the dataset

In [ ]:
gametime_master['Game_Conf'] = gametime_master.apply(lambda x: in_conf_game(x['Team Code'], 
                                                                            x['Opp Code'], 
                                                                            get_season_str_yr(x['Date'])), axis=1)

In [ ]:
gametime_master['Game_NZ_Capacity'] = gametime_master.apply(lambda x: 
                                                                      get_norm_stadium_capacity(x['Game Code']), 
                                                                      axis=1)

### Add season win/loss stats for each team to the dataset

In [ ]:
gametime_master['Team_WTD'] = gametime_master.apply(lambda x: season_wins_to_date(x['Team Code'], x['Date']), axis=1)
gametime_master['Team_LTD'] = gametime_master.apply(lambda x: season_losses_to_date(x['Team Code'], x['Date']), axis=1)
gametime_master['Team_RTD'] = gametime_master.apply(lambda x: season_record_to_date(x['Team Code'], 
                                                                                    x['Date']), axis=1)
gametime_master['Team_CRTD'] = gametime_master.apply(lambda x: conf_record_to_date(x['Team Code'], 
                                                                                    x['Date']), axis=1)

### Add season win/loss stats for each team's opponent to the dataset

In [ ]:
gametime_master['Opp_WTD'] = gametime_master.apply(lambda x: season_wins_to_date(x['Opp Code'], x['Date']), axis=1)
gametime_master['Opp_LTD'] = gametime_master.apply(lambda x: season_losses_to_date(x['Opp Code'], x['Date']), axis=1)
gametime_master['Opp_RTD'] = gametime_master.apply(lambda x: season_record_to_date(x['Opp Code'], 
                                                                                   x['Date']), axis=1)
gametime_master['Opp_CRTD'] = gametime_master.apply(lambda x: conf_record_to_date(x['Opp Code'], 
                                                                                   x['Date']), axis=1)

### Write the gametime_master file to a .csv file in the output directory

In [3]:
gametime_master.to_csv('./output/gametime_master.csv')

NameError: name 'gametime_master' is not defined

### Input the training set

In [4]:
gametime = pd.read_csv('./output/gametime_master.csv')
gametime = gametime.drop('Unnamed: 1', axis=1)
gametime.rename(columns = {'Unnamed: 0':'Season'}, inplace = True)
gametime.set_index('Season')
gametime.head()

,Season,Game Code,Date,Team Code,Opp Code,Won,Game_Home,Game_Conf,Game_NZ_Capacity,Team_WTD,Team_LTD,Team_RTD,Team_CRTD,Opp_WTD,Opp_LTD,Opp_RTD,Opp_CRTD
0,2005,86016420050901,2005-09-01,164,86,1,1,0,0.372090,0,0,0.0,0.0,0,0,0.0,0.0
1,2005,128064820050901,2005-09-01,648,128,1,1,0,0.746505,0,0,0.0,0.0,0,0,0.0,0.0
2,2005,204014020050901,2005-09-01,140,204,1,1,0,0.353485,0,0,0.0,0.0,0,0,0.0,0.0
3,2005,295075420050901,2005-09-01,754,295,1,1,0,0.372090,0,0,0.0,0.0,0,0,0.0,0.0
4,2005,428071920050901,2005-09-01,719,428,0,1,0,0.375671,0,0,0.0,0.0,0,0,0.0,0.0


### Check for missing values and show them if applicable

In [5]:
missing_val_count_by_column = (gametime.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
df = gametime
null_data = df[df.isnull().any(axis=1)]
null_data

Series([], dtype: int64)


,Season,Game Code,Date,Team Code,Opp Code,Won,Game_Home,Game_Conf,Game_NZ_Capacity,Team_WTD,Team_LTD,Team_RTD,Team_CRTD,Opp_WTD,Opp_LTD,Opp_RTD,Opp_CRTD


### Train the Decision Tree, Random Forest and XGBoost models

In [6]:
features = gametime.columns[6:]

y = gametime.Won
X = gametime[features]

train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=1)

DT_model = DecisionTreeClassifier(random_state=1)
RF_model = RandomForestClassifier(random_state=1)
XG_model = XGBClassifier()

DT_model.fit(train_X, train_y)
RF_model.fit(train_X, train_y)
XG_model.fit(train_X, train_y)

DT_preds = DT_model.predict(val_X)
RF_preds = RF_model.predict(val_X)
XG_preds = XG_model.predict(val_X)

DT_scores = accuracy_score(val_y, DT_preds)
RF_scores = accuracy_score(val_y, RF_preds)
XG_scores = accuracy_score(val_y, XG_preds)

# Print the scores for each model
print('DT - Accuracy: ' + str(DT_scores))
print('RF - Accuracy: ' + str(RF_scores))
print('XG - Accuracy: ' + str(XG_scores))

/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


DT - Accuracy: 0.6398891966759003
RF - Accuracy: 0.6808864265927977
XG - Accuracy: 0.7163434903047091


### DT, RF & XG Model results
Baseline Model (no features)<br>
DT - Accuracy: 0.6617728531855955<br>
RF - Accuracy: 0.6698060941828254<br>
XG - Accuracy: 0.7038781163434903 <br><br>
Added Game Conf & NZ Stadium Capacity<br>
DT - Accuracy: 0.6373961218836565<br>
RF - Accuracy: 0.6648199445983379<br>
XG - Accuracy: 0.7083102493074792<br><br>
Removed Home, replaced True Home with Game_Home, streamlined full model trainer<br>
DT - Accuracy: 0.6238227146814405<br>
RF - Accuracy: 0.6559556786703601<br>
XG - Accuracy: 0.7102493074792243<br><br>
Added Conf Records for Team and Opp Team<br>
DT - Accuracy: 0.6398891966759003<br>
RF - Accuracy: 0.6808864265927977<br>
XG - Accuracy: 0.7163434903047091<br><br>

### Train the Keras Classifer Deep Learning Model

In [7]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# specify prediction variable and features
features = gametime.columns[6:]
y = gametime.Won
X = gametime[features]

# save input dimensions for first layer of model
X_dim = X.shape[1]

# encode class values as integers
#encoder = LabelEncoder()
#encoder.fit(Y)
#encoded_Y = encoder.transform(Y)

# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(X_dim, input_dim=X_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=100)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, y, cv=kfold)
print("50 epochs, 100 batchsize, 5 splits")
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

end_time=time.time()
print("total time: ",end_time-start_time)

Epoch 1/50
11552/11552 [==============================] - 3s 241us/step - loss: 0.6569 - acc: 0.6530
Epoch 2/50
11552/11552 [==============================] - 1s 68us/step - loss: 0.5925 - acc: 0.6975
Epoch 3/50
11552/11552 [==============================] - 1s 73us/step - loss: 0.5746 - acc: 0.7038
Epoch 4/50
11552/11552 [==============================] - 1s 71us/step - loss: 0.5683 - acc: 0.7106
Epoch 5/50
11552/11552 [==============================] - 1s 69us/step - loss: 0.5652 - acc: 0.7097
Epoch 6/50
11552/11552 [==============================] - 1s 75us/step - loss: 0.5640 - acc: 0.7092
Epoch 7/50
11552/11552 [==============================] - 1s 73us/step - loss: 0.5636 - acc: 0.7088
Epoch 8/50
11552/11552 [==============================] - 1s 70us/step - loss: 0.5625 - acc: 0.7078
Epoch 9/50
11552/11552 [==============================] - 1s 68us/step - loss: 0.5628 - acc: 0.7089
Epoch 10/50
11552/11552 [==============================] - 1s 73us/step - loss: 0.5615 - acc: 0.708

11552/11552 [==============================] - 1s 64us/step - loss: 0.5544 - acc: 0.7141
Epoch 32/50
11552/11552 [==============================] - 1s 59us/step - loss: 0.5547 - acc: 0.7142
Epoch 33/50
11552/11552 [==============================] - 1s 60us/step - loss: 0.5545 - acc: 0.7138
Epoch 34/50
11552/11552 [==============================] - 1s 63us/step - loss: 0.5547 - acc: 0.7150
Epoch 35/50
11552/11552 [==============================] - 1s 64us/step - loss: 0.5539 - acc: 0.7163
Epoch 36/50
11552/11552 [==============================] - 1s 50us/step - loss: 0.5539 - acc: 0.7134
Epoch 37/50
11552/11552 [==============================] - 1s 61us/step - loss: 0.5543 - acc: 0.7151
Epoch 38/50
11552/11552 [==============================] - 1s 61us/step - loss: 0.5544 - acc: 0.7158
Epoch 39/50
11552/11552 [==============================] - 1s 60us/step - loss: 0.5540 - acc: 0.7162
Epoch 40/50
11552/11552 [==============================] - 1s 62us/step - loss: 0.5546 - acc: 0.7133
Ep

11552/11552 [==============================] - 1s 63us/step - loss: 0.5623 - acc: 0.7094
Epoch 12/50
11552/11552 [==============================] - 1s 63us/step - loss: 0.5632 - acc: 0.7112
Epoch 13/50
11552/11552 [==============================] - 1s 61us/step - loss: 0.5613 - acc: 0.7104
Epoch 14/50
11552/11552 [==============================] - 1s 61us/step - loss: 0.5617 - acc: 0.7103
Epoch 15/50
11552/11552 [==============================] - 1s 61us/step - loss: 0.5616 - acc: 0.7122
Epoch 16/50
11552/11552 [==============================] - 1s 60us/step - loss: 0.5612 - acc: 0.7105
Epoch 17/50
11552/11552 [==============================] - 1s 57us/step - loss: 0.5614 - acc: 0.7091
Epoch 18/50
11552/11552 [==============================] - 1s 59us/step - loss: 0.5611 - acc: 0.7091
Epoch 19/50
11552/11552 [==============================] - 1s 54us/step - loss: 0.5610 - acc: 0.7097
Epoch 20/50
11552/11552 [==============================] - 1s 63us/step - loss: 0.5612 - acc: 0.7106
Ep

11552/11552 [==============================] - 1s 59us/step - loss: 0.5575 - acc: 0.7139
Epoch 42/50
11552/11552 [==============================] - 1s 58us/step - loss: 0.5575 - acc: 0.7152
Epoch 43/50
11552/11552 [==============================] - 1s 63us/step - loss: 0.5582 - acc: 0.7155
Epoch 44/50
11552/11552 [==============================] - 1s 62us/step - loss: 0.5576 - acc: 0.7148
Epoch 45/50
11552/11552 [==============================] - 1s 48us/step - loss: 0.5577 - acc: 0.7149
Epoch 46/50
11552/11552 [==============================] - 1s 66us/step - loss: 0.5583 - acc: 0.7130
Epoch 47/50
11552/11552 [==============================] - 1s 63us/step - loss: 0.5577 - acc: 0.7154
Epoch 48/50
11552/11552 [==============================] - ETA: 0s - loss: 0.5578 - acc: 0.715 - 1s 63us/step - loss: 0.5576 - acc: 0.7158
Epoch 49/50
11552/11552 [==============================] - 1s 65us/step - loss: 0.5577 - acc: 0.7162
Epoch 50/50
2888/2888 [==============================] - 0s 170us

#### KerasClassifier Results
Baseline Model (no features)<br>
50 epochs, 100 batchsize, 5 splits
Results: 70.92% (1.02%)
total time:  196.68944692611694 <br>
Added Game Conf & NZ Stadium Capacity<br>
50 epochs, 100 batchsize, 5 splits
Results: 70.74% (1.06%)
total time:  237.4298951625824<br>
Removed Home, replaced True Home with Game_Home, streamlined full model trainer<br>
50 epochs, 100 batchsize, 5 splits
Results: 71.00% (1.13%)
total time:  212.06504821777344<br>
Added Conf Records for Team and Opp Team<br>
50 epochs, 100 batchsize, 5 splits
Results: 71.49% (1.02%)
total time:  270.62656712532043<br>